# Prétraitement des Données - Challenge Titanic

Ce notebook vous guide à travers les étapes essentielles de prétraitement des données pour le challenge Titanic. L'objectif est de transformer les données brutes en caractéristiques exploitables par les algorithmes de machine learning.

## Objectifs du prétraitement:
- Gérer les valeurs manquantes de manière appropriée
- Créer de nouvelles caractéristiques pertinentes
- Encoder les variables catégorielles
- Préparer les données pour l'entraînement des modèles

In [ ]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Pour le prétraitement
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Configuration de l'affichage
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
sns.set(font_scale=1.2)

# Pour afficher les graphiques dans le notebook
%matplotlib inline

# Pour afficher toutes les colonnes
pd.set_option('display.max_columns', None)

## 1. Chargement des données

Commençons par charger les données d'entraînement et de test.

In [ ]:
# Charger les données
train_data = pd.read_csv('/workspaces/titanicML/Data/train.csv')
test_data = pd.read_csv('/workspaces/titanicML/Data/test.csv')

# Sauvegarde des identifiants de passagers pour la soumission finale
test_passenger_ids = test_data['PassengerId']

# Aperçu des données
print(f"Données d'entraînement: {train_data.shape[0]} lignes et {train_data.shape[1]} colonnes")
print(f"Données de test: {test_data.shape[0]} lignes et {test_data.shape[1]} colonnes")

# Vérifier les valeurs manquantes
missing_train = train_data.isnull().sum()
missing_test = test_data.isnull().sum()

missing_train_percent = (missing_train / len(train_data)) * 100
missing_test_percent = (missing_test / len(test_data)) * 100

missing_df = pd.DataFrame({
    'Train - Valeurs manquantes': missing_train,
    'Train - Pourcentage (%)': missing_train_percent,
    'Test - Valeurs manquantes': missing_test,
    'Test - Pourcentage (%)': missing_test_percent
})

display(missing_df[(missing_df['Train - Valeurs manquantes'] > 0) | 
                 (missing_df['Test - Valeurs manquantes'] > 0)].sort_values('Train - Pourcentage (%)', ascending=False))

## 2. Combinaison temporaire des ensembles de données pour le prétraitement

Pour assurer une cohérence dans le prétraitement, nous allons combiner temporairement les ensembles d'entraînement et de test.

In [ ]:
# Séparer les variables cibles avant la combinaison
train_targets = train_data['Survived']

# Marquer les données d'entraînement et de test pour pouvoir les séparer plus tard
train_data['is_train'] = 1
test_data['is_train'] = 0

# Combiner les données pour le prétraitement
combined_data = pd.concat([train_data.drop('Survived', axis=1), test_data], axis=0).reset_index(drop=True)

print(f"Dimensions des données combinées: {combined_data.shape[0]} lignes et {combined_data.shape[1]} colonnes")
print(f"Dont {train_data.shape[0]} lignes d'entraînement et {test_data.shape[0]} lignes de test")

# Aperçu des données combinées
display(combined_data.head())

## 3. Gestion des valeurs manquantes

### 3.1 Imputation de l'âge manquant

Pour imputer les âges manquants, nous allons utiliser une approche basée sur le titre et la classe du passager.

In [ ]:
# Extraction du titre à partir du nom
combined_data['Title'] = combined_data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

# Afficher les titres uniques
print("Titres uniques trouvés:", combined_data['Title'].unique())
print("\nFréquence des titres:")
display(combined_data['Title'].value_counts())

# Regrouper les titres moins fréquents
title_mapping = {
    'Mr': 'Mr',
    'Miss': 'Miss',
    'Mrs': 'Mrs',
    'Master': 'Master',
    'Dr': 'Rare',
    'Rev': 'Rare',
    'Col': 'Rare',
    'Major': 'Rare',
    'Mlle': 'Miss',
    'Countess': 'Rare',
    'Ms': 'Miss',
    'Lady': 'Rare',
    'Jonkheer': 'Rare',
    'Don': 'Rare',
    'Dona': 'Rare',
    'Mme': 'Mrs',
    'Capt': 'Rare',
    'Sir': 'Rare'
}

combined_data['Title'] = combined_data['Title'].map(title_mapping)

# Vérifier la distribution des titres après regroupement
display(combined_data['Title'].value_counts())

# Calculer l'âge médian par titre et classe
age_by_title_class = combined_data.groupby(['Title', 'Pclass'])['Age'].median()
print("\nÂge médian par titre et classe:")
display(age_by_title_class)

# Fonction pour imputer l'âge manquant
def impute_age(row):
    if pd.isnull(row['Age']):
        return age_by_title_class[(row['Title'], row['Pclass'])]
    else:
        return row['Age']

# Appliquer la fonction d'imputation
combined_data['Age'] = combined_data.apply(impute_age, axis=1)

# Vérifier que toutes les valeurs d'âge ont été imputées
print(f"\nValeurs d'âge manquantes après imputation: {combined_data['Age'].isnull().sum()}")

# Si certains titres rares n'ont pas de médiane d'âge dans certaines classes, utiliser la médiane globale par titre
if combined_data['Age'].isnull().sum() > 0:
    age_by_title = combined_data.groupby('Title')['Age'].transform('median')
    combined_data.loc[combined_data['Age'].isnull(), 'Age'] = age_by_title[combined_data['Age'].isnull()]

print(f"Valeurs d'âge manquantes après la seconde imputation: {combined_data['Age'].isnull().sum()}")

# Imputer toute valeur restante avec la médiane globale
if combined_data['Age'].isnull().sum() > 0:
    combined_data['Age'].fillna(combined_data['Age'].median(), inplace=True)

print(f"Valeurs d'âge manquantes après imputation finale: {combined_data['Age'].isnull().sum()}")

# Visualiser la distribution de l'âge avant et après imputation
plt.figure(figsize=(12, 6))

# Distribution originale (utiliser les données d'entraînement d'origine)
plt.subplot(1, 2, 1)
sns.histplot(train_data['Age'].dropna(), kde=True, bins=30, color='blue')
plt.title('Distribution originale de l\'âge (sans valeurs manquantes)')
plt.xlabel('Âge')
plt.ylabel('Fréquence')

# Distribution après imputation
plt.subplot(1, 2, 2)
sns.histplot(combined_data.loc[combined_data['is_train'] == 1, 'Age'], kde=True, bins=30, color='green')
plt.title('Distribution de l\'âge après imputation')
plt.xlabel('Âge')
plt.ylabel('Fréquence')

plt.tight_layout()
plt.show()

### 3.2 Imputation des autres valeurs manquantes

Gérons maintenant les valeurs manquantes pour le port d'embarquement et le tarif.

In [ ]:
# Imputation du port d'embarquement (Embarked)
# Vérifier les valeurs manquantes
print(f"Valeurs manquantes pour 'Embarked': {combined_data['Embarked'].isnull().sum()}")

if combined_data['Embarked'].isnull().sum() > 0:
    # Afficher les lignes avec des valeurs manquantes
    print("\nPassagers avec port d'embarquement manquant:")
    display(combined_data[combined_data['Embarked'].isnull()])
    
    # Imputer avec le mode (valeur la plus fréquente)
    most_common_port = combined_data['Embarked'].mode()[0]
    combined_data['Embarked'].fillna(most_common_port, inplace=True)
    print(f"\nPort d'embarquement '{most_common_port}' utilisé pour imputer les valeurs manquantes.")

# Imputation du tarif (Fare)
print(f"\nValeurs manquantes pour 'Fare': {combined_data['Fare'].isnull().sum()}")

if combined_data['Fare'].isnull().sum() > 0:
    # Afficher les lignes avec des valeurs manquantes
    print("\nPassagers avec tarif manquant:")
    display(combined_data[combined_data['Fare'].isnull()])
    
    # Imputer avec la médiane par classe
    median_fare_by_class = combined_data.groupby('Pclass')['Fare'].median()
    
    for pclass, fare in median_fare_by_class.items():
        combined_data.loc[(combined_data['Fare'].isnull()) & (combined_data['Pclass'] == pclass), 'Fare'] = fare
    
    print("\nTarifs médians par classe utilisés pour l'imputation:")
    display(median_fare_by_class)

# Vérifier que toutes les valeurs manquantes ont été imputées
missing_after = combined_data.isnull().sum()
print("\nValeurs manquantes après imputation:")
display(missing_after[missing_after > 0])

## 4. Création de nouvelles caractéristiques

Créons maintenant des caractéristiques qui pourraient être utiles pour la prédiction.

In [ ]:
# 4.1 Taille de la famille et passagers voyageant seuls
combined_data['FamilySize'] = combined_data['SibSp'] + combined_data['Parch'] + 1  # +1 pour inclure le passager
combined_data['IsAlone'] = (combined_data['FamilySize'] == 1).astype(int)

# Créer des catégories de tailles de famille
combined_data['FamilyGroup'] = pd.cut(
    combined_data['FamilySize'],
    bins=[0, 1, 4, 11],
    labels=['None', 'Small', 'Large']
)

# 4.2 Extraction des informations sur la cabine
# Identifier si le passager a une cabine enregistrée
combined_data['HasCabin'] = combined_data['Cabin'].notna().astype(int)

# Extraire le pont de la cabine (première lettre)
combined_data['Deck'] = combined_data['Cabin'].str.slice(0, 1)
combined_data['Deck'].fillna('U', inplace=True)  # U pour "Unknown" (inconnu)

# 4.3 Créer des groupes d'âge
combined_data['AgeGroup'] = pd.cut(
    combined_data['Age'],
    bins=[0, 12, 18, 35, 60, 100],
    labels=['Child', 'Teenager', 'Young Adult', 'Adult', 'Senior']
)

# 4.4 Créer des groupes de tarifs
combined_data['FareGroup'] = pd.qcut(
    combined_data['Fare'],
    q=5,
    labels=['Very Low', 'Low', 'Medium', 'High', 'Very High']
)

# 4.5 Créer une caractéristique pour le nom de famille
# Extraire le nom de famille
combined_data['Surname'] = combined_data['Name'].str.split(',').str[0]

# 4.6 Créer une caractéristique pour identifier les femmes mariées
combined_data['IsMarriedWoman'] = ((combined_data['Title'] == 'Mrs') | 
                                 (combined_data['Title'] == 'Mme')).astype(int)

# Aperçu des nouvelles caractéristiques
print("Aperçu des nouvelles caractéristiques:")
columns_to_display = ['PassengerId', 'Title', 'Age', 'AgeGroup', 'FamilySize', 
                     'FamilyGroup', 'IsAlone', 'HasCabin', 'Deck', 'FareGroup', 'IsMarriedWoman']
display(combined_data[columns_to_display].head(10))

## 5. Encodage des variables catégorielles

Encodons les variables catégorielles pour les rendre utilisables par les algorithmes de machine learning.

In [ ]:
# 5.1 Encodage du sexe (Sex)
combined_data['Sex_Code'] = combined_data['Sex'].map({'female': 1, 'male': 0})

# 5.2 Encodage du port d'embarquement (Embarked)
# One-hot encoding pour Embarked
embarked_dummies = pd.get_dummies(combined_data['Embarked'], prefix='Embarked')
combined_data = pd.concat([combined_data, embarked_dummies], axis=1)

# 5.3 Encodage du titre (Title)
# One-hot encoding pour Title
title_dummies = pd.get_dummies(combined_data['Title'], prefix='Title')
combined_data = pd.concat([combined_data, title_dummies], axis=1)

# 5.4 Encodage du pont (Deck)
# One-hot encoding pour Deck
deck_dummies = pd.get_dummies(combined_data['Deck'], prefix='Deck')
combined_data = pd.concat([combined_data, deck_dummies], axis=1)

# 5.5 Encodage des groupes d'âge (AgeGroup)
# One-hot encoding pour AgeGroup
age_group_dummies = pd.get_dummies(combined_data['AgeGroup'], prefix='AgeGroup')
combined_data = pd.concat([combined_data, age_group_dummies], axis=1)

# 5.6 Encodage des groupes de taille de famille (FamilyGroup)
# One-hot encoding pour FamilyGroup
family_group_dummies = pd.get_dummies(combined_data['FamilyGroup'], prefix='FamilyGroup')
combined_data = pd.concat([combined_data, family_group_dummies], axis=1)

# 5.7 Encodage des groupes de tarifs (FareGroup)
# One-hot encoding pour FareGroup
fare_group_dummies = pd.get_dummies(combined_data['FareGroup'], prefix='FareGroup')
combined_data = pd.concat([combined_data, fare_group_dummies], axis=1)

# Afficher les nouvelles dimensions du DataFrame
print(f"Dimensions du DataFrame après encodage: {combined_data.shape[0]} lignes et {combined_data.shape[1]} colonnes")
print(f"Nouvelles colonnes créées: {combined_data.shape[1] - 18}")  # 18 colonnes d'origine

# Aperçu des premières colonnes du DataFrame
display(combined_data.head())

## 6. Sélection et préparation des caractéristiques finales

Sélectionnons maintenant les caractéristiques pertinentes pour l'entraînement des modèles.

In [ ]:
# 6.1 Définir les caractéristiques à conserver pour le modèle
features_to_use = [
    # Caractéristiques numériques
    'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'FamilySize',
    
    # Encodage binaire
    'Sex_Code', 'IsAlone', 'HasCabin', 'IsMarriedWoman',
    
    # Variables encodées (one-hot)
    'Embarked_C', 'Embarked_Q', 'Embarked_S',
    'Title_Mr', 'Title_Mrs', 'Title_Miss', 'Title_Master', 'Title_Rare',
    
    # Ajout des caractéristiques de pont (sélectionner les plus pertinentes)
    'Deck_A', 'Deck_B', 'Deck_C', 'Deck_D', 'Deck_E', 'Deck_F', 'Deck_G', 'Deck_U',
    
    # Groupes d'âge
    'AgeGroup_Child', 'AgeGroup_Teenager', 'AgeGroup_Young Adult', 'AgeGroup_Adult', 'AgeGroup_Senior',
    
    # Groupes de famille
    'FamilyGroup_None', 'FamilyGroup_Small', 'FamilyGroup_Large'
    
    # Nous excluons les groupes de tarif car ils pourraient être redondants avec 'Fare'
]

# Afficher le nombre de caractéristiques sélectionnées
print(f"Nombre de caractéristiques sélectionnées: {len(features_to_use)}")

# 6.2 Création des ensembles d'entraînement et de test finaux
# Séparer les ensembles d'entraînement et de test
train_processed = combined_data.loc[combined_data['is_train'] == 1, features_to_use].reset_index(drop=True)
test_processed = combined_data.loc[combined_data['is_train'] == 0, features_to_use].reset_index(drop=True)

print(f"Ensemble d'entraînement prétraité: {train_processed.shape[0]} lignes et {train_processed.shape[1]} colonnes")
print(f"Ensemble de test prétraité: {test_processed.shape[0]} lignes et {test_processed.shape[1]} colonnes")

# Aperçu des données prétraitées
print("\nAperçu des données d'entraînement prétraitées:")
display(train_processed.head())

# 6.3 Normalisation des caractéristiques numériques
numerical_features = ['Age', 'Fare', 'FamilySize', 'SibSp', 'Parch']

# Créer un scaler pour les caractéristiques numériques
scaler = StandardScaler()

# Ajuster le scaler sur les données d'entraînement et transformer les deux ensembles
train_processed[numerical_features] = scaler.fit_transform(train_processed[numerical_features])
test_processed[numerical_features] = scaler.transform(test_processed[numerical_features])

# Vérifier les statistiques après normalisation
print("\nStatistiques des caractéristiques numériques après normalisation (ensemble d'entraînement):")
display(train_processed[numerical_features].describe())

## 7. Sauvegarde des données prétraitées

Sauvegardons les ensembles de données prétraités pour les utiliser lors de la phase de modélisation.

In [ ]:
# Créer un dictionnaire avec les objets importants
preprocessed_data = {
    'train_features': train_processed,
    'train_target': train_targets,
    'test_features': test_processed,
    'test_passenger_ids': test_passenger_ids,
    'feature_names': features_to_use,
    'scaler': scaler  # Pour appliquer la même normalisation aux données futures
}

# Chemin de sauvegarde
import os
from sklearn.externals import joblib

# Créer le dossier pour les données prétraitées s'il n'existe pas
preprocessed_dir = '/workspaces/titanicML/Data/preprocessed'
if not os.path.exists(preprocessed_dir):
    os.makedirs(preprocessed_dir)

# Sauvegarder les données prétraitées
joblib.dump(preprocessed_data, os.path.join(preprocessed_dir, 'preprocessed_data.pkl'))

print(f"Données prétraitées sauvegardées dans {preprocessed_dir}")

# Sauvegarder également au format CSV pour une inspection facile
train_processed_with_target = train_processed.copy()
train_processed_with_target['Survived'] = train_targets

train_processed_with_target.to_csv(os.path.join(preprocessed_dir, 'train_processed.csv'), index=False)
test_processed.to_csv(os.path.join(preprocessed_dir, 'test_processed.csv'), index=False)

print("Fichiers CSV également générés pour une inspection facile des données prétraitées")

## 8. Conclusions et prochaines étapes

Dans ce notebook, nous avons:

1. **Géré les valeurs manquantes**:
   - Imputé les âges manquants en fonction du titre et de la classe
   - Imputé le port d'embarquement manquant avec la valeur la plus fréquente
   - Imputé les tarifs manquants avec la médiane par classe

2. **Créé de nouvelles caractéristiques**:
   - Taille de la famille et indicateur pour les passagers voyageant seuls
   - Extraction des titres à partir des noms
   - Information sur la présence d'une cabine et le pont
   - Groupes d'âge et de tarifs

3. **Encodé les variables catégorielles**:
   - Encodage binaire pour le sexe
   - One-hot encoding pour les autres variables catégorielles

4. **Normalisé les caractéristiques numériques**:
   - Application du StandardScaler pour mettre les variables à la même échelle

Les données sont maintenant prêtes pour la phase de modélisation, où nous testerons différents algorithmes de classification pour prédire la survie des passagers du Titanic.

### Prochaines étapes:
- Entraîner différents modèles de classification (Random Forest, XGBoost, etc.)
- Optimiser les hyperparamètres des modèles
- Évaluer les performances des modèles
- Sélectionner le meilleur modèle pour la prédiction finale